In [3]:
import numpy as np
import os
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn import model_selection, preprocessing as pp
from sklearn.metrics import classification_report, mean_absolute_error
from sklearn.svm import SVR
from sklearn.externals import joblib
import sys

In [5]:
#location of the typical meteorological year (tmy) dataset
#download the file from nrel if interested
filename_tmy = os.path.join("tmy3MSPairport.csv")

So, the dataset we're using is the Typical Meteorological Year dataset from NREL.  

The TMY3 dataset is a year's worth of hourly meteorological data, by location.
It is derived from the 1961-1990 and 1991-2005 National Solar Radiation Database  archives.  Each month of the year is the month that is determined to be "most typical" from all of the data in the archive.  You can read more about the TMY datasets here: https://rredc.nrel.gov/solar/old_data/nsrdb/1991-2005/tmy3/.

There are a few limitations of this dataset:

1. It is a "typical" meteorological year, therefore, it won't help in dealing with extremes and worst case scenerios.
2. It is relatively small at 8760 points.  On a related note, because it is just one year of data, the annual seasonality of average solar irradiance is seen as an upwards trend until (roughly) the summer solstice, followed by a downward trend until (roughly) the winter solstice rather than the roughly sinusoidal signal that would be observed over multiple years.

In spite of these drawbacks, I believe this is a good dataset to start with.  It is currently being used in modelling monthly generation for use in Power Purchase Agreements (PPA), is decently comprehensive and is organized by location for over 1000 sites.  In addition, this was my first project, and one year's worth of data seemed to be a bit easier to chew than 10 or more times that.


In [6]:
tmy_df = pd.read_csv(filename_tmy, header=1, low_memory=False)

In [8]:
tmy_df.head()

,Date (MM/DD/YYYY),Time (HH:MM),ETR (W/m^2),ETRN (W/m^2),GHI (W/m^2),GHI source,GHI uncert (%),DNI (W/m^2),DNI source,DNI uncert (%),...,Alb (unitless),Alb source,Alb uncert (code),Lprecip depth (mm),Lprecip quantity (hr),Lprecip source,Lprecip uncert (code),PresWth (METAR code),PresWth source,PresWth uncert (code)
0,1/1/2004,1:00,0,0,0,2,0,0,2,0,...,0,F,8,0,1,D,9,0,C,8
1,1/1/2004,2:00,0,0,0,2,0,0,2,0,...,0,F,8,0,1,D,9,0,C,8
2,1/1/2004,3:00,0,0,0,2,0,0,2,0,...,0,F,8,0,1,D,9,0,C,8
3,1/1/2004,4:00,0,0,0,2,0,0,2,0,...,0,F,8,0,1,D,9,0,C,8
4,1/1/2004,5:00,0,0,0,2,0,0,2,0,...,0,F,8,0,1,D,9,0,C,8


Here's a look at the first few rows of the dataset.  

It isn't shown here, but each month has a different year since each month has been "selected" as being the most typical. 

Again, hourly data.

The columns we will be concerned with mostly in this are ETR (Extraterrestrial Radiation), DNI (Direct Normal Irradiance) and DHI (Diffuse Horizontal Irradiance).  In short, ETR indicates the amount of solar radiation above the weather (extraterrestrial).  DNI, or beam irradiance, is the sunlight hitting a reflector directly, at a perpendicular angle.  DHI, or diffuse irradiance is the photons that are scattered due to particulate matter in the atmosphere.

NOTE: System generation is very close to being linearly related to the total irradiance (DNI + DHI), so the sum of those together, rather than evaluating each individually is probably a better path forward in the future.

The weather data we will be concerned with is the TotCld (cloud coverage), Dry-bulb (temperature), and RHum (relative humidity).

In [9]:
tmy_weather = tmy_df[['ETR (W/m^2)', 'GHI (W/m^2)', 'DNI (W/m^2)', 'DHI (W/m^2)', 
'TotCld (tenths)', 'Dry-bulb (C)', 'RHum (%)']]

Now, we will convert the dataframe to an array and split it into DNI and DHI datasets, as we will be training and evaluating the model on each of those separately.  We will also hold 20% of the data back in order to validate our model later.

While it is not implemented in this project, in other projects of mine, I've used ETR as a sort of "night time" flag.  Sometimes there can be 0 irradiance during the day because of thick clouds, but ETR would still be positive.  However, if ETR is 0, we can tell the model that there is no possible irradiance, and set the prediction to 0.  


In [51]:
tmy_weather_a = tmy_weather.values
#X values are all rows in ETR (sort of like time of day), Clouds, Temp, Relative Humidity
X = tmy_weather_a[:, [0, 4, 5, 6]]
#Y values are just DNI and DHI: direct and diffuse irradiance
Y = tmy_weather_a[:, [2, 3]]
#split once further:
DNI_Y = Y[:, 0]
DHI_Y = Y[:, 1]
#Note that both models will use the same X, or input data

#20% held back for validation
validation_size = 0.20
#initialize random seed for reproducibility
DNI_seed = 7
DHI_seed = 5
#train_test_split the data randomly into training and testing sets
X_train, X_val, DNI_Y_train, DNI_Y_val = model_selection.train_test_split(X, DNI_Y, test_size=validation_size, random_state=DNI_seed)
X_train, X_val, DHI_Y_train, DHI_Y_val = model_selection.train_test_split(X, DHI_Y, test_size=validation_size, random_state=DHI_seed)


So, anyone following along who knows a thing or two about data science / machine learning will realize that randomizing the data is not a good idea, since this is quite clearly a time series forecasting problem.  It's a place to start in this case.

Next, we'll standardize all of the data.  Using sklearn's StandardScaler(), we get data with a mean of 0 and standard deviation of 1.

In [56]:
X_standardizer = pp.StandardScaler()
DNI_Y_standardizer = pp.StandardScaler()
DHI_Y_standardizer = pp.StandardScaler()

#note that the 1 dimensional y array needs to be reshaped for the function to 
#operate properly
DNI_Y_train = np.array(DNI_Y_train).reshape((len(DNI_Y_train), 1))
DHI_Y_train = np.array(DHI_Y_train).reshape((len(DHI_Y_train), 1))

X_train_scaled = X_standardizer.fit_transform(X_train)
DNI_Y_train_scaled = DNI_Y_standardizer.fit_transform(DNI_Y_train)
DHI_Y_train_scaled = DHI_Y_standardizer.fit_transform(DHI_Y_train)




Now, it's time to train and evaluate the model.  I went with a Support Vector Regression model, after finding it had the best results, but admittedly, at that time, I didn't really know what that meant.  In later posts I will explore both a time series analysis with an ARIMA model as well as a LSTM neural network, and likely some other classical models such as decision trees, linear regression, logistic regression, etc.

We'll use 10 fold cross validation with mean absolute error and R squared as our evaluation metrics

In [25]:
scores = ['neg_mean_absolute_error', 'r2']
kfolds = 10
model = SVR()

In [31]:
def trainer(X_data, Y_data, seed, kfolds):
    kfold = model_selection.KFold(n_splits=kfolds, random_state=seed)
    #evaluate for each scoring metric
    for i, method in enumerate(scores):
        #note, .ravel() is used on the Y_data since we reshaped it earlier in order
        #to fit into the scaler. Ravel brings it back to its original dimensionality.
        cv_results = model_selection.cross_val_score(model, 
                                                     X_data,
                                                     Y_data.ravel(),
                                                     cv=kfold,
                                                     scoring=method)
        print(scores[i] + ' mean: ', cv_results.mean())
        print(scores[i] + ' stdev: ', cv_results.std())

print('DNI: ')       
trainer(X_train, DNI_Y_train, DNI_seed, kfolds)
print('DHI: ')
trainer(X_train, DHI_Y_train, DHI_seed, kfolds)

DNI: 
neg_mean_absolute_error mean:  -170.48493870006075
neg_mean_absolute_error stdev:  11.6201121481729
r2 mean:  -0.3746117178631775
r2 stdev:  0.031553171110808555
DHI: 
neg_mean_absolute_error mean:  -64.42495128473288
neg_mean_absolute_error stdev:  1.7290716717993322
r2 mean:  -0.31608475507340883
r2 stdev:  0.021089446201420142


On average, our DNI predictions were off by 170.5 W/m^2 and our DHI values by 64.4.  The R squared value does not indicate a particularly good fit.  

Much of this could be likely be improved simply by accounting for night time, which we are not currently doing, and will be shown more explicitly shortly.  

Next, we'll run this model on our validation dataset, which is "unseen" to the model.  This is a sort of test run, before we send the model out into the real world, although this model won't be going anywhere anytime soon.

In [36]:
#scale validation data
X_val_scaled = X_standardizer.fit_transform(X_val)
DNI_Y_val = np.array(DNI_Y_val).reshape((len(DNI_Y_val), 1))
DHI_Y_val = np.array(DHI_Y_val).reshape((len(DHI_Y_val), 1))
DNI_Y_val_scaled = DNI_Y_standardizer.fit_transform(DNI_Y_val)
DHI_Y_val_scaled = DHI_Y_standardizer.fit_transform(DHI_Y_val)

#define, fit models
SVM_DNI = SVR()
SVM_DHI = SVR()
SVM_DNI.fit(X_val_scaled, DNI_Y_val_scaled.ravel())
SVM_DHI.fit(X_val_scaled, DHI_Y_val_scaled.ravel())


#make predictions
predictions_DNI = SVM_DNI.predict(X_val_scaled)
predictions_DHI = SVM_DHI.predict(X_val_scaled)

predictions_DNI = np.array(predictions_DNI).reshape((len(predictions_DNI), 1))
predictions_DHI = np.array(predictions_DHI).reshape((len(predictions_DHI), 1))

#invert the predictions back to normal scale
predictions_DNI_inverted = DNI_Y_standardizer.inverse_transform(predictions_DNI)
predictions_DHI_inverted = DHI_Y_standardizer.inverse_transform(predictions_DHI)
X_validation_inverted = X_standardizer.fit_transform(X_val_scaled)

In [38]:
print("MAE for DNI predictions: ")
print(mean_absolute_error(DNI_Y_val, predictions_DNI_inverted))
print("MAE for DHI predictions: ")
print(mean_absolute_error(DHI_Y_val, predictions_DHI_inverted))

MAE for DNI predictions: 
182.0650567127503
MAE for DHI predictions: 
15.302799766543252


In [46]:
#take a look at the actual values:
pred_df = pd.DataFrame({
    'DNI: ': predictions_DNI_inverted.tolist(),
    'DHI: ': predictions_DHI_inverted.tolist()
})
pred_df.head(50)



,DHI:,DNI:
0,[43.61546279705216],[28.971766623758896]
1,[157.5277992893514],[28.922890622505804]
2,[7.498861590925991],[31.66642194249488]
3,[358.45724160839194],[28.94301905330488]
4,[2.167354164775354],[28.994463593440514]
5,[8.216939259254865],[28.747784394130775]
6,[7.774946717283768],[28.68043531553826]
7,[7.607161200108457],[27.329066109501667]
8,[16.569607786379763],[29.309344546088397]
9,[255.40240817826472],[29.745066928385853]


Since the data was randomized during the train test split, there's only so much we can extract from these predictions.

That's going to be the end of this quick little intro to my humble start in this realm.  Some improvements that will be documented in the near future:

1. Taking into account the temporal structure of the data.
2. More Visualization of the data in general
3. Different models (ARIMA, decision trees, LSTM neural networks)
4. More Data - Pulling from the NSRDB rather than just using TMY.  This will give us access to years and years of data rather than just the one, combined set.
5. Making predictions on real out of sample data - shortly, I will show my program for pulling from the National Digital Forecast Database API, but there are many, many weather databases to choose from all with different strengths and weaknesses.

I'm sure there's more to improve, but this is a good jumping off point I think, and stumbling through the project six months ago really helped me get an idea of what machine learning is and what it can do.  Thanks for reading, and I'll see you next time :).